## Домашнее задание 3

    Используя RandomForestClassifier и еще 2 метода МО предсказать будущий ABC класс на основе любых фичей



In [40]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [8]:
data = pd.read_excel(r'/Users/olgahromova/Desktop/УчЕБА/DataSet_w_NA (1).xlsx', sheet_name='Испорченные факты').dropna()
data

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10288,2018-169929,Нет,2019-03-25 00:00:00,2019-03-28 00:00:00,Первый класс,CLNT0000182,PROD0000869,19950.0,5.0,0.0,5187.00,1,14,6
10289,2018-169978,Нет,2019-06-24 00:00:00,2019-06-28 00:00:00,Стандарт,CLNT0000575,PROD0000293,135882.0,2.0,0.1,24156.80,2,11,11
10290,2018-169978,Нет,2019-06-24 00:00:00,2019-06-28 00:00:00,Стандарт,CLNT0000575,PROD0001131,7188.0,3.0,0.2,2425.95,2,11,4
10291,2018-169999,Нет,2018-12-19 00:00:00,2018-12-26 00:00:00,Стандарт,CLNT0000013,PROD0000549,5784.0,3.0,0.2,1879.80,2,11,13


ABC анализ

In [9]:
df = pd.pivot_table(data, 
                    values=[
                        'Продажи, руб', 
                        'Продажи, шт',
                        'Повторение заказа', 
                        'Маржинальная прибыль',
                        'Повторение товара'
                            ],
                    index=["Факты.Товар ID"], 
                    aggfunc={
                            'Продажи, шт': [np.median, np.sum], 
                            'Продажи, руб': np.sum, 
                            'Повторение заказа': np.sum, 
                            'Маржинальная прибыль': np.sum
                            })
df.columns = df.columns.map('_'.join)
df = df.reset_index()

tot_inc = df['Продажи, руб_sum'].sum()
df['share'] = df['Продажи, руб_sum']/tot_inc
df.sort_values(by='share', inplace=True, ascending=False)
df = df.assign(share_ad = df.share.cumsum())

df.loc[(df['share_ad'] <= 0.8), 'ABC'] = 'A'
df.loc[(df['share_ad'] > 0.8) & (df['share_ad'] <= 95), 'ABC'] = 'B'
df.loc[(df['share_ad'] > 0.95), 'ABC'] = 'C'


Модель предсказания категории товара

In [45]:
def  model(method, X, Y, name: str):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
    mod = method
    mod.fit(X_train, Y_train)
    predict = mod.predict(X_test)
    result = X_test.assign(ABC_pred=predict, ABC=Y)
    print(f"Prediction of {name}:", f'accuracy:{metrics.accuracy_score(Y_test, predict)}')
    return display(result)
features = df[[
              'Маржинальная прибыль_sum', 
                'Повторение заказа_sum', 
                'Продажи, шт_sum'
                ]]
rand_f = model(method=RandomForestClassifier(),X=features , Y=df.ABC, name='random forest classifier')
k_means = model(method=KNeighborsClassifier(), X=features, Y=df.ABC, name='k-means classifier')
supp_vec = model(method=svm.SVC(), X=features, Y=df.ABC, name='support vectors classifier')

Prediction of random forest classifier: accuracy:0.774798927613941


,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, шт_sum",ABC_pred,ABC
879,16893.5,21,25.0,C,C
326,15370.0,12,22.0,C,C
609,19709.8,24,18.0,C,C
1710,41688.0,3,4.0,C,C
1777,-25187.4,9,14.0,B,C
...,...,...,...,...,...
20,27619.0,24,23.0,C,C
581,3370.0,8,5.0,C,C
1780,4786.8,2,2.0,C,C
1318,-57338.2,9,13.0,B,A


Prediction of k-means classifier: accuracy:0.7640750670241286


,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, шт_sum",ABC_pred,ABC
1516,16588.8,11,14.0,C,C
1261,13140.4,8,4.0,C,C
191,22294.0,16,25.0,C,C
1679,20451.6,6,16.0,C,C
638,137664.3,27,34.0,B,A
...,...,...,...,...,...
737,-171547.8,21,26.0,A,A
1141,187110.0,20,13.0,B,A
1709,5032.8,10,13.0,C,C
1858,-82476.9,6,3.0,A,C


Prediction of support vectors classifier: accuracy:0.7372654155495979


,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, шт_sum",ABC_pred,ABC
858,2183.85,24,41.0,C,C
99,27986.40,11,24.0,C,C
196,-86470.60,30,28.0,C,A
1381,11662.20,17,14.0,C,C
1110,304021.50,11,14.0,A,A
...,...,...,...,...,...
1792,15291.00,3,3.0,C,C
1216,4642.60,16,18.0,C,C
1191,149937.50,15,21.0,B,A
763,14026.20,16,19.0,C,C
